# Homework 2

In order to properly view the plot, open this file in nbviewer at:

[https://nbviewer.org/github/gbulg/Data-analysis-class/blob/main/HW2/hw2.ipynb](https://nbviewer.org/github/gbulg/Data-analysis-class/blob/main/HW2/hw2.ipynb)

## 1. Create Database and fill it

I will use the twitter sentiment dataset that is mentioned in the task. Originally it is a table with just 3 columns:
*   ItemID -- an index column
*   Sentiment -- a boolean value column, indicating the positive/negative sentiment of the tweet
*   SentimentText -- a text value, containing the text of the tweet



However, analyzing tweet texts we can often find mentions of other users, or some hashtags. I will find such mentions and hashtags in tweets and based on that create the following database with 5 tables:
*   `tweets` table, the original table untouched
*   `users` table, with 2 columns: `user_id`, `username` (only users that were at least once mentioned will be here)
*   `hashtags` table, with 2 columns: `hashtag_id`, `hashtag`
*   `mentions` table, with 3 columns: `mention_id`, `tweet_id` and `mention_id` (an entry here means that tweet with `tweet_id` mentions a user with `user_id`)
*   `hashtag_usage` table, with 3 columns: `hashtag_usage_id`, `tweet_id` and `hashtag_id` (an entry here means that tweet with `tweet_id` contains a hashtag with `hashtag_id`)




Import sqlite and create tables:

In [1]:
import sqlite3

# connecting to the database
conn = sqlite3.connect('twitter.db')

# creating a cursor object we will be sending our queries to
c = conn.cursor()

In [2]:
# delete everything if there was something
c.execute("DROP TABLE IF EXISTS tweets")
c.execute("DROP TABLE IF EXISTS users")
c.execute("DROP TABLE IF EXISTS hashtags")
c.execute("DROP TABLE IF EXISTS mentions")
c.execute("DROP TABLE IF EXISTS hashtag_usage")

# create tables
c.execute("CREATE TABLE tweets(tweet_id INTEGER PRIMARY KEY AUTOINCREMENT, sentiment BOOL, text TEXT)")
c.execute("CREATE TABLE users(user_id INTEGER PRIMARY KEY AUTOINCREMENT, username STRING)")
c.execute("CREATE TABLE hashtags(hashtag_id INTEGER PRIMARY KEY AUTOINCREMENT, hashtag STRING)")
c.execute("CREATE TABLE mentions(mention_id INTEGER PRIMARY KEY AUTOINCREMENT, tweet_id INT, user_id INT)")
c.execute("CREATE TABLE hashtag_usage(hashtag_usage_id INTEGER PRIMARY KEY AUTOINCREMENT, tweet_id INT, hashtag_id INT)")

Now load the dataset:

In [3]:
import pandas as pd
import requests

In [4]:
dataset_url = "https://raw.githubusercontent.com/vineetdhanawat/twitter-sentiment-analysis/master/datasets/Sentiment%20Analysis%20Dataset%20100000.csv"

twitter_dataset = pd.read_csv(dataset_url, sep=",", encoding="ISO-8859-1")

In [5]:
twitter_dataset.columns = ['tweet_id', 'sentiment', 'text']
twitter_dataset.head(15)

,tweet_id,sentiment,text
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


Fill in the tables:



In [6]:
import re

# iterate over rows
for _, tweet in twitter_dataset.iterrows():
  # insert in the main table
  c.execute("INSERT INTO tweets VALUES (?, ?, ?)", (tweet['tweet_id'], tweet['sentiment'], tweet['text']))

  # find all mentions using regular expressions
  mentions = re.findall("(^|[^@\w])@(\w{1,15})", tweet['text'])
  for _, user in mentions:
    c.execute("SELECT user_id FROM users WHERE username = ?", (user,))
    res = c.fetchall()
    if res == []:
      c.execute("INSERT INTO users (username) VALUES (?)", (user,))
      c.execute("SELECT user_id FROM users WHERE username = ?", (user,))
      user_id = c.fetchall()[0][0]
    else:
      user_id = res[0][0]
    c.execute("INSERT INTO mentions (tweet_id, user_id) VALUES (?, ?)", (tweet['tweet_id'], user_id))

  # find hashtags using regular expressions
  hashtags = list(set(re.findall(r"#(\w+)", tweet['text'])))
  for hashtag in hashtags:
    c.execute("SELECT hashtag_id FROM hashtags WHERE hashtag = ?", (hashtag,))
    res = c.fetchall()
    if res == []:
      c.execute("INSERT INTO hashtags (hashtag) VALUES (?)", (hashtag,))
      c.execute("SELECT hashtag_id FROM hashtags WHERE hashtag = ?", (hashtag,))
      hashtag_id = c.fetchall()[0][0]
    else:
      hashtag_id = res[0][0]
    c.execute("INSERT INTO hashtag_usage (tweet_id, hashtag_id) VALUES (?, ?)", (tweet['tweet_id'], hashtag_id))

In [7]:
# commiting the changes
conn.commit()

Download the data back from DB to pandas dataframes:

In [8]:
tweets_df = pd.DataFrame(pd.read_sql_query("SELECT * FROM tweets", conn), columns = ['tweet_id', 'sentiment', 'text'])
users_df = pd.DataFrame(pd.read_sql_query("SELECT * FROM users", conn), columns = ['user_id', 'username'])
hashtags_df = pd.DataFrame(pd.read_sql_query("SELECT * FROM hashtags", conn), columns = ['hashtag_id', 'hashtag'])
mentions_df = pd.DataFrame(pd.read_sql_query("SELECT * FROM mentions", conn), columns = ['mention_id', 'tweet_id', 'user_id'])
hashtag_usage_df = pd.DataFrame(pd.read_sql_query("SELECT * FROM hashtag_usage", conn), columns = ['hashtag_usage_id', 'tweet_id', 'hashtag_id'])

In [16]:
# closing the database
conn.close()

Now we can close the DB connection. Check that all dataframes are fine:

In [22]:
tweets_df[100:105]

,tweet_id,sentiment,text
100,101,0,no pavel tonight &lt;Tigersfan &gt;
101,102,0,not a cool night.
102,103,1,oh thank you!
103,104,1,pleased
104,105,0,probably guna get off soon since no one is t...


In [23]:
users_df[100:105]

,user_id,username
100,101,Lakers
101,102,PERFEKTnCHANCE
102,103,Standing_Stones
103,104,311
104,105,ERiceOnTheBeat


In [24]:
mentions_df[100:105]

,mention_id,tweet_id,user_id
100,101,344,100
101,102,358,101
102,103,359,102
103,104,360,103
104,105,363,104


# 2. Manipulating DataFrames and Plotting


## 2.1 Average sentiment for hashtag

It seems to me that there should be some hashtags that are used primarily in high sentiment tweets, and some, in opposite, generally used in low sentiment tweets. To check this, let's calculate for each hashtag the average sentiment of tweets with this hashtag. I will add a new column `average_sentiment` to the `hashtags_df` dataframe.

In [80]:
tweet_lists = hashtag_usage_df.groupby("hashtag_id", as_index=False).agg({"tweet_id": list})
tweet_lists

,hashtag_id,tweet_id
0,1,"[132, 4368, 4377, 4378, 4379, 4380, 4381, 4382..."
1,2,"[133, 4532, 4533, 4534, 4535, 5596, 5597, 5598..."
2,3,[134]
3,4,"[135, 283, 385, 651, 2678, 2679, 2910, 4820, 6..."
4,5,"[136, 89618]"
...,...,...
2742,2743,[99689]
2743,2744,[99737]
2744,2745,[99979]
2745,2746,[99979]


In [81]:
average_sentiments = [None] * hashtags_df.shape[0]
for i in hashtags_df.hashtag_id:
  tweet_list = tweet_lists[tweet_lists.hashtag_id == i].tweet_id.values[0] # list of tweets with this hashtag
  average_sentiments[i-1] = tweets_df[tweets_df.tweet_id.isin(tweet_list)].sentiment.mean(axis=0)

hashtags_df['average_sentiment'] = average_sentiments
hashtags_df

,hashtag_id,hashtag,average_sentiment,usage_count
0,1,3turnoffwords,0.428571,35
1,2,asylm,0.476190,21
2,3,poemsunder140,1.000000,1
3,4,squarespace,0.278689,122
4,5,Susan,0.000000,2
...,...,...,...,...
2742,2743,australia,0.000000,1
2743,2744,youdabest,1.000000,1
2744,2745,Spain,1.000000,1
2745,2746,Diabetes,1.000000,1


## 2.2 Hashtag usage count

Now i think that some hashtags may be used only once, and thus have an average sentiment equal to 0 or 1, but it is not very interesting. So let's add a `usage_count` column to filter them out.

In [83]:
hashtags_df['usage_count'] = list(hashtag_usage_df.groupby("hashtag_id", as_index=False).count().hashtag_usage_id)

In [84]:
hashtags_df[hashtags_df.usage_count > 10]

,hashtag_id,hashtag,average_sentiment,usage_count
0,1,3turnoffwords,0.428571,35
1,2,asylm,0.476190,21
3,4,squarespace,0.278689,122
8,9,iphone,0.250000,12
12,13,robotpickuplines,0.909091,11
...,...,...,...,...
982,983,tweetdeck,0.153846,13
1002,1003,unfollowdiddy,0.416667,12
1039,1040,WhyITweet,0.727273,11
1961,1962,NoUndiesSunday,1.000000,16


A list of high sentiment hashtags:

In [97]:
hashtags_df[(hashtags_df.usage_count > 10) & (hashtags_df.average_sentiment > 0.8)].sort_values(by='usage_count', ascending=False)

,hashtag_id,hashtag,average_sentiment,usage_count
26,27,followfriday,0.947253,455
64,65,musicmonday,0.968421,190
66,67,FollowFriday,0.948718,117
317,318,FF,0.904348,115
393,394,goodsex,0.883333,60
318,319,ff,0.865385,52
100,101,3hotwords,0.965517,29
94,95,marsiscoming,0.807692,26
706,707,mw2,0.863636,22
125,126,andyclemmensen,0.909091,22


Now we can use a bar plot to visualise it.

In [ ]:
import plotly.offline as pyo
import plotly.express as px
import plotly.io as pio

pyo.init_notebook_mode(connected=True)
pio.renderers.default = 'notebook_connected'

In [108]:
fig = px.bar(hashtags_df[hashtags_df.usage_count > 10].sort_values(by='average_sentiment', ascending=False),
             x='hashtag', y='average_sentiment', color='usage_count')
fig.show()

## 2.3 Mention count for users

Let's add the same column to the users dataframe

In [103]:
users_df['mention_count'] = list(mentions_df.groupby("user_id", as_index=False).count().mention_id)

users_df[users_df.mention_count > 30]

,user_id,username,mention_count
151,152,billbeckett,33
163,164,billyraycyrus,119
467,468,bradiewebbstack,125
795,796,Beverleyknight,33
917,918,amandapalmer,68
...,...,...,...
46841,46842,ColorblindFish,53
46969,46970,comeagainjen,45
47021,47022,ComedyQueen,48
49506,49507,CrunchyK,46


## 2.4 Identify users that are mentioned only in positive/only in negative context
Let's add two columns to `users_df` -- `only_pos` and `only neg`. They will have 0/1 values. `only_pos` equals to 1 means that all tweets, where this user is mentioned, have sentiment 1.

In [122]:
# a list of tweets with 0 sentiment
neg_tweets = list(tweets_df[tweets_df.sentiment == 0].tweet_id)

# a list of users, mentioned in neg tweets.
mentioned_in_neg = list(mentions_df[mentions_df.tweet_id.isin(neg_tweets)].user_id) 

# analogously with pos
pos_tweets = list(tweets_df[tweets_df.sentiment == 1].tweet_id)
mentioned_in_pos = list(mentions_df[mentions_df.tweet_id.isin(pos_tweets)].user_id) 

In [126]:
# set only_pos to 0 if is in mentioned_in_neg, 1 otherwise
users_df['only_pos'] = [int(not user in mentioned_in_neg) for user in users_df.user_id]

# analogously
users_df['only_neg'] = [int(not user in mentioned_in_pos) for user in users_df.user_id]

users_df

,user_id,username,mention_count,only_pos,only_neg
0,1,shaunamanu,1,0,1
1,2,ginaaa,1,1,0
2,3,Spiral_galaxy,4,0,1
3,4,YMPtweet,4,0,1
4,5,rupturerapture,1,0,1
...,...,...,...,...,...
51350,51351,Cupcake,1,0,1
51351,51352,cupcake__,1,1,0
51352,51353,CuPcAkE_2120,1,0,1
51353,51354,Cupcake_Dollie,1,1,0


Now let's look at the amounts of different categories of users:

In [134]:
amounts_of_users = users_df[['only_pos', 'only_neg', 'username']].groupby(['only_pos', 'only_neg'], as_index=False).count()
amounts_of_users.columns = ['only_pos', 'only_neg', 'amount']
amounts_of_users['percentage'] = amounts_of_users['amount'] / users_df.shape[0]

amounts_of_users

,only_pos,only_neg,amount,percentage
0,0,0,6943,0.135196
1,0,1,17708,0.344815
2,1,0,26704,0.519988


And now the same, but only for users that were mentioned more than 5 times:

In [136]:
amounts_of_users = users_df[users_df.mention_count > 5][['only_pos', 'only_neg', 'username']].groupby(['only_pos', 'only_neg'], as_index=False).count()
amounts_of_users.columns = ['only_pos', 'only_neg', 'amount']
amounts_of_users['percentage'] = amounts_of_users['amount'] / users_df[users_df.mention_count > 5].shape[0]

amounts_of_users

,only_pos,only_neg,amount,percentage
0,0,0,1560,0.910152
1,0,1,31,0.018086
2,1,0,123,0.071762


We see that among users that are mentioned at least several times, the vast majority are being mention in positive and negative tweets as well.

Let's get a list of users with highest number of mentions while being mentioned in positive tweets only:

In [137]:
users_df[users_df.only_pos == 1].sort_values(by='mention_count', ascending=False).head(10)

,user_id,username,mention_count,only_pos,only_neg
1475,1476,1sweetwhirl,33,1,0
4044,4045,Banksyart,22,1,0
2333,2334,combustiblesong,19,1,0
21217,21218,AshleyOlsen7,17,1,0
22602,22603,aussie_ali,15,1,0
45188,45189,ClaireBoyles,15,1,0
36620,36621,CarinaK,14,1,0
10747,10748,alexderossi,13,1,0
22598,22599,ausi1,12,1,0
635,636,Annjj,12,1,0
